<a href="https://colab.research.google.com/github/RodriMenendez/BIO_LM_SURF/blob/main/practice/Classification_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and Importing

In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import lightning as L

## Data Preprocessing

In [3]:
path = '/content/drive/MyDrive/SURF_2023/Practice'

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_data = torchvision.datasets.CIFAR10(root=path+'/cifar10', train=True, transform=transform, download=True)

batch_size = 32
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_data = torchvision.datasets.CIFAR10(root=path+'/cifar10', train=False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

classes = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

100%|██████████| 170498071/170498071 [00:01<00:00, 101834584.36it/s]


Extracting /content/drive/MyDrive/SURF_2023/Practice/cifar10/cifar-10-python.tar.gz to /content/drive/MyDrive/SURF_2023/Practice/cifar10
Files already downloaded and verified


## Model Definition

In [4]:
class Classification(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.pretrained_model = pretrained_model
        self.layer1 = nn.Linear(1000, 1000)
        self.layer2 = nn.Linear(1000, 10)

    def probabilities(self, x):
        output =  self.pretrained_model(x)
        output = F.relu(self.layer1(output))
        output = F.softmax(self.layer2(output), dim=1)

        return output

    def forward(self, x):
        output = self.probabilities(x)
        output = torch.argmax(output, dim=1)

        return output

In [5]:
resnet_model = torchvision.models.resnet18(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 106MB/s]


In [6]:
model = Classification(resnet_model)

## Lightning Code

In [7]:
class CIFARModule(L.LightningModule):
    def __init__(self, model, lr=1e-3, momentum=0.9):
        super().__init__()
        self.model = model
        self.loss_module = nn.CrossEntropyLoss()
        self.lr = lr
        self.momentum = momentum

    def forward(self, imgs):
        return torch.argmax(self.model(imgs), dim=1)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr, momentum=self.momentum)
        return optimizer

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        one_hot_labels = F.one_hot(labels, 10)

        probs = self.model.probabilities(inputs)
        loss = self.loss_module(probs, one_hot_labels.float())
        preds = self.model(inputs)
        acc = (preds == labels).float().mean()

        # log accuracy and loss
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        self.log("train_loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)
        acc = (preds == labels).float().mean()

        self.log("test_acc", acc)

In [8]:
autoencoder = CIFARModule(model)

In [9]:
trainer = L.pytorch.Trainer(accelerator='auto', max_epochs=4)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model=autoencoder, train_dataloaders=trainloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name        | Type             | Params
-------------------------------------------------
0 | model       | Classification   | 12.7 M
1 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
12.7 M    Trainable params
0         Non-trainable params
12.7 M    Total params
50.802    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name        | Type             | Params
-------------------------------------------------
0 | model       | Classification   | 12.7 M
1 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
12.7 M    Trainable params
0         Non-trainable params
12.7 M    Total params
50.802    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=4` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


In [11]:
test_result = trainer.test(autoencoder, dataloaders=testloader, verbose=False)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

In [12]:
test_result

[{'test_acc': 0.9347000122070312}]